# Using Agate to resolve all
those lingering questions about Central Wyoming hay production data
I'm in Wyoming for a weeklong vacation at the folks' place, and the question came up: Is Fremont County ~really~ in the top 50 U.S. counties for alfalfa hay production? We can use [`Agate`](http://agate.readthedocs.io) to analyze [USDA/NASS data](https://nass.usda.gov) to find out for sure.

First, we download a CSV with [this cut of data](https://quickstats.nass.usda.gov/results/39B79FD0-B6CD-3739-8274-9828B86B6637) -- county-level alfalfa hay production tonnage figures from the government's quinquennial agriculture census -- and save it as `hay.csv`. I found this approach easier than using [the agency's API](https://quickstats.nass.usda.gov/api), but YMMV.

In [211]:
%%bash
head hay.csv

"Program","Year","Period","Week Ending","Geo Level","State","State ANSI","Ag District","Ag District Code","County","County ANSI","Zip Code","Region","watershed_code","Watershed","Commodity","Data Item","Domain","Domain Category","Value","CV (%)"
"CENSUS","2012","YEAR","","COUNTY","ALABAMA","01","BLACK BELT","40","AUTAUGA","001","","","00000000","","HAY","HAY, ALFALFA - PRODUCTION, MEASURED IN TONS","TOTAL","NOT SPECIFIED","300","57.2"
"CENSUS","2012","YEAR","","COUNTY","ALABAMA","01","BLACK BELT","40","DALLAS","047","","","00000000","","HAY","HAY, ALFALFA - PRODUCTION, MEASURED IN TONS","TOTAL","NOT SPECIFIED","144","57.2"
"CENSUS","2012","YEAR","","COUNTY","ALABAMA","01","BLACK BELT","40","ELMORE","051","","","00000000","","HAY","HAY, ALFALFA - PRODUCTION, MEASURED IN TONS","TOTAL","NOT SPECIFIED"," (D)","(D)"
"CENSUS","2012","YEAR","","COUNTY","ALABAMA","01","BLACK BELT","40","GREENE","063","","","00000000","","HAY","HAY, ALFALFA - PRODUCTION, MEASURED IN TONS","TOTAL","NOT SPECIFIED

### Extract only the columns we care about

In [212]:
%%bash
csvcut -c 2,7,11,6,10,20 hay.csv > hay-cut.csv
head hay-cut.csv

Year,State ANSI,County ANSI,State,County,Value
2012,01,001,ALABAMA,AUTAUGA,300
2012,01,047,ALABAMA,DALLAS,144
2012,01,051,ALABAMA,ELMORE, (D)
2012,01,063,ALABAMA,GREENE, (D)
2012,01,065,ALABAMA,HALE,74
2012,01,085,ALABAMA,LOWNDES,440
2012,01,087,ALABAMA,MACON, (D)
2012,01,091,ALABAMA,MARENGO,70
2012,01,101,ALABAMA,MONTGOMERY,44


### Fire up Agate

In [213]:
from collections import OrderedDict
import agate

In [214]:
# USDA ag censuses
CENSUS_YEARS = ['1997', '2002', '2007', '2012']

# define the column types
col_types = {
    'Year': agate.Text(),
    'State ANSI': agate.Text(),
    'County ANSI': agate.Text(),
    'State': agate.Text(),
    'County': agate.Text(),
    'Value': agate.Text()
}

In [215]:
# load the csv into an agate table
alfalfa_production = agate.Table.from_csv('hay-cut.csv', column_types = col_types)

In [216]:
alfalfa_production.print_table()

|-------+------------+-------------+---------+------------+--------|
|  Year | State ANSI | County ANSI | State   | County     | Value  |
|-------+------------+-------------+---------+------------+--------|
|  2012 | 01         | 001         | ALABAMA | AUTAUGA    | 300    |
|  2012 | 01         | 047         | ALABAMA | DALLAS     | 144    |
|  2012 | 01         | 051         | ALABAMA | ELMORE     | (D)    |
|  2012 | 01         | 063         | ALABAMA | GREENE     | (D)    |
|  2012 | 01         | 065         | ALABAMA | HALE       | 74     |
|  2012 | 01         | 085         | ALABAMA | LOWNDES    | 440    |
|  2012 | 01         | 087         | ALABAMA | MACON      | (D)    |
|  2012 | 01         | 091         | ALABAMA | MARENGO    | 70     |
|  2012 | 01         | 101         | ALABAMA | MONTGOMERY | 44     |
|  2012 | 01         | 105         | ALABAMA | PERRY      | 27     |
|  2012 | 01         | 119         | ALABAMA | SUMTER     | 178    |
|  2012 | 01         | 003        

### Remove null values and recast the 'Value' column as a number
Values for counties with zero alfalfa production == " (D)".

In [217]:
alfalfa_production_no_nulls_sorted = alfalfa_production.where(
    lambda row: row['Value'].strip() != '(D)'
).compute([
    ('value_as_int', agate.Formula(agate.Number(), lambda row: int(row['Value'].replace(',', ''))))
]).exclude('Value').order_by('value_as_int', reverse=True)

In [218]:
alfalfa_production_no_nulls_sorted.print_table()

|-------+------------+-------------+------------+-----------+---------------|
|  Year | State ANSI | County ANSI | State      | County    | value_as_int  |
|-------+------------+-------------+------------+-----------+---------------|
|  2002 | 06         | 025         | CALIFORNIA | IMPERIAL  |    1,586,024  |
|  2012 | 06         | 025         | CALIFORNIA | IMPERIAL  |    1,173,325  |
|  1997 | 06         | 025         | CALIFORNIA | IMPERIAL  |    1,021,810  |
|  2007 | 06         | 025         | CALIFORNIA | IMPERIAL  |      976,999  |
|  2002 | 53         | 025         | WASHINGTON | GRANT     |      898,626  |
|  2002 | 06         | 029         | CALIFORNIA | KERN      |      803,933  |
|  2007 | 53         | 025         | WASHINGTON | GRANT     |      752,332  |
|  2002 | 06         | 107         | CALIFORNIA | TULARE    |      718,581  |
|  1997 | 06         | 029         | CALIFORNIA | KERN      |      714,922  |
|  1997 | 53         | 025         | WASHINGTON | GRANT     |   

### Create tables for each year of data

In [219]:
data_by_year = {}

def get_rank_by_year(year):
    return alfalfa_production_no_nulls_sorted.where(
        lambda row: row['Year'] == year
    ).compute([
        ('rank', agate.Rank('value_as_int', reverse=True))
    ])

for year in CENSUS_YEARS:
    data_by_year[year] = get_rank_by_year(year)
    
data_by_year = OrderedDict(sorted(data_by_year.items(), key=lambda t: int(t[0])))

#### Get Fremont County data for each year

In [220]:
for year in data_by_year:
    fc_data = data_by_year[year].where(
        lambda row: row['County ANSI'] == '013' and row['State ANSI'] == '56'
    )
    print(
        year,
        "/",
        '{0:,}'.format(fc_data.rows[0]['value_as_int']),
        "tons / ranked",
        fc_data.rows[0]['rank'],
        "out of",
        '{0:,}'.format(len(data_by_year[data].rows)),
        "producing counties"
    )

1997 / 219,046 tons / ranked 29 out of 2,319 producing counties
2002 / 184,041 tons / ranked 47 out of 2,319 producing counties
2007 / 275,524 tons / ranked 22 out of 2,319 producing counties
2012 / 278,957 tons / ranked 17 out of 2,319 producing counties


# ~~ Verdict: True ~~